
# wps_ca

A process that runs the [ca.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/CA.R) function from ClimDown.

Constructed Analogue (CA) downscaling algorithm: Starts by spatially aggregating high-resolution gridded observations up to the scale of a GCM. Then it proceeds to bias correcting the GCM based on those observations. Finally, it conducts the search for temporal analogues. This involves taking each timestep in the GCM and searching for the top 30 closest timesteps in the gridded observations. For each of the 30 closest "analogue" timesteps, CA records the integer number of the timestep (indices) and a weight for each of the analogues.


In [1]:
from birdy import WPSClient
from pkg_resources import resource_filename
from wps_tools.testing import get_target_url
from tempfile import NamedTemporaryFile
from datetime import date
from urllib.request import urlopen
import requests
import os

from wps_tools.output_handling import rda_to_vector, auto_construct_outputs
from chickadee.utils import test_analogues

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [3]:
chickadee = WPSClient(url)

### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.ca?

Signature:
chickadee.ca(
    gcm_file,
    obs_file=None,
    varname=None,
    num_cores='4',
    loglevel='INFO',
    units_bool=True,
    n_pr_bool=True,
    tasmax_units='celsius',
    tasmin_units='celsius',
    pr_units='kg m-2 d-1',
    max_gb=1.0,
    start_date=datetime.date(1971, 1, 1),
    end_date=datetime.date(2005, 12, 31),
    num_analogues='30',
    delta_days='45',
    trimmed_mean=0.0,
    tol=0.1,
    output_file='output.rda',
    vector_name='output_vector',
    output_formats=None,
)
Docstring:
Constructed Analogue (CA) downscaling algorithm

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of GCM simulations
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
varname : string
    Name of the NetCDF variable to downscale (e.g. 'tasmax')
num_cores : {'1', '2', '3', '4'}positiveInteger
    The num

### Run CA process

In [5]:
with NamedTemporaryFile(
    suffix=".rda", prefix="weights_", dir="/tmp", delete=True
) as output_file:
    output = chickadee.ca(
        gcm_file=resource_filename("tests", "/data/tiny_gcm.nc"), 
        obs_file=resource_filename("tests", "/data/tiny_obs.nc"), 
        varname="tasmax", 
        end_date=date(1972, 12, 31),
        output_file=output_file.name,
        vector_name="analogues_output"
    )
url = output.get()[0]

Access the output with rda_to_vector() or auto_construct_outputs() from wps_tools.output_handling

In [6]:
# NBVAL_IGNORE_OUTPUT
rda_to_vector(url, "analogues_output")

indices,[RTYPES.VECSXP]
weights,[RTYPES.VECSXP]


In [7]:
# NBVAL_IGNORE_OUTPUT
auto_construct_outputs(output.get())

[R object with classes: ('list',) mapped to:
 [ListSexpVector, ListSexpVector]
   indices: <class 'rpy2.rinterface.ListSexpVector'>
   <rpy2.rinterface.ListSexpVector object at 0x7fd213cb43c0> [RTYPES.VECSXP]
   weights: <class 'rpy2.rinterface.ListSexpVector'>
   <rpy2.rinterface.ListSexpVector object at 0x7fd213cc6040> [RTYPES.VECSXP]]

### Test for expected output

In [8]:
test_analogues(
    url, "analogues_output", "analogues.rda", "analogues"
)